# Project 3

## Getting Insights into Images and their Metadata


Analysis will include:
...

In [21]:
%matplotlib inline
import scipy
import scipy.io
import numpy as np
import torch
import torchvision
from torchvision import models, transforms
from sklearn import metrics
import sqlite3
import matplotlib
from matplotlib import pyplot as plt
from PIL import Image

In [22]:
vgg16 = torchvision.models.vgg16(pretrained=True) # very slow command

C:\Users\saraw\miniconda3\envs\mlfds\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\saraw\miniconda3\envs\mlfds\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


funny, for me that command took maybe half a second? /sara

In [23]:
# vgg16.features is a series of modules.
# we can chain an input through the modules via the forward method.
phi = vgg16.features.forward

def preprocess_image(img):

    # commented out lines are (probably) unnecessary? I hope?
    transform = transforms.Compose([
        #transforms.Resize(256), # or 224
        #transforms.CenterCrop(224),
        transforms.ToTensor(),
        #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    batch = transform(img)
    # batch = torch.unsqueeze(batch, 0)
    return batch 

def fetch_img(img_index:int, healthy:bool=True):
    if healthy:
        return Image.open(f"Apple___healthy/image ({img_index}).JPG")
    else:
        return Image.open(f"Apple___Black_rot/image ({img_index}).JPG")

def img_features(img_index:int, healthy:bool=True, flatten=False):
    res = phi(preprocess_image(fetch_img(img_index, healthy=healthy)))
    if flatten:
        res = torch.flatten(res)
    return res

phi_out_shape = img_features(1, healthy=True).shape
phi_out_shape


torch.Size([512, 8, 8])

The guessing process consists of the following pipeline of functions:

fetch_img -> preprocess_image -> phi (aka our neural network) -> flatten -> guess

where guess requires w to be calculated, which is the normalized flattened difference of mean phi outputs of healthy and unhealthy images.


In [24]:
# first we need to calculate the means for both healthy and unhealthy image features
def mean_features(indices, healthy=True, flatten = True):
    mean = None
    for x in indices:
        if mean is None:
            mean = img_features(x, healthy=healthy, flatten=flatten)
        else:
            mean += img_features(x, healthy=healthy, flatten=flatten)

    return mean / len(indices)

training_indices = range(1, 50) # first few images as training set. Can always be changed later. (Now)

mean_healthy = mean_features(training_indices, healthy=True)
mean_rotted = mean_features(training_indices, healthy=False)

w = mean_rotted - mean_healthy
w = w / torch.linalg.norm(w) 

def guess(x): # takes a flattened tensor of image features
    return torch.dot(w, x) # magnitude of projection onto line from mean to mean

def img_guess(img_index, healthy=True): 
    return guess(img_features(img_index, healthy, flatten=True))

We want to test the guessing with some examples:

In [26]:
# see if we can correctly identify the first healthy/unhealthy leaves not in the training set
test_healthy = img_guess(51, healthy=True)
test_rotted = img_guess(51, healthy=False)

mean_healthy_position = guess(mean_healthy)
mean_rotted_position = guess(mean_rotted)
# smaller values imply higher confidence
true_neg = torch.abs(test_healthy - mean_healthy_position)
true_pos = torch.abs(test_rotted - mean_rotted_position)
false_neg = torch.abs(test_rotted - mean_healthy_position)
false_pos = torch.abs(test_healthy - mean_rotted_position)
print(true_neg)
print(true_pos)
print(false_neg)
print(false_pos)
# smaller values imply higher confidence

tensor(3.2134, grad_fn=<AbsBackward0>)
tensor(5.3685, grad_fn=<AbsBackward0>)
tensor(40.4772, grad_fn=<AbsBackward0>)
tensor(31.8954, grad_fn=<AbsBackward0>)


interpretation: Our model can quite accurately determine if the image is rotted based on just 20 images of training data. (now 50)

In [31]:
#write some kind of decision function

test_size = 10 # classify 10 random images that were not in the test set
true_labels = np.tile(np.array([1, 0]), test_size)
estimated = []

for t in range(test_size):
    # rotten has exactly 1000 entrys
    # healty has 1645 or so pictures, i don't know for sure though (I'm dyslexic, could be anything like 1456 or 1654 or whatever)
    # so lets just say 1000
    rand_test = np.random.randint(51, 1000)
    
    test_healthy = img_guess(rand_test, healthy=True)
    test_rotted = img_guess(rand_test, healthy=False)
    # decide on label (I know its ugly code but I'm tired for now)
    if torch.abs(test_healthy - mean_healthy_position) < torch.abs(test_healthy - mean_rotted_position):
        estimated.append(1)
    else:
        estimated.append(0)
    if torch.abs(test_rotted - mean_rotted_position) < torch.abs(test_rotted - mean_healthy_position):
        estimated.append(0)
    else:
        estimated.append(1)

print('AUC of ROC: ', metrics.roc_auc_score(true_labels, estimated))

AUC of ROC:  0.9


weee, thats decent enough

In [46]:
# something something sensitivity
y = torch.mean(img_features(21, healthy=True, flatten=True)) # take mean so its a scalar
x = preprocess_image(fetch_img(21, healthy=True))
x.requires_grad = True
torch.autograd.grad(y, x, create_graph=True)[0]

RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.